In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from glob import glob
from tqdm import tqdm
import shutil
import nibabel as nib
from subprocess import run
import random
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
df=pd.DataFrame(columns=['class','data','label1','label2'])
count=0
class_list=['ADC','DWI']
for i in range(len(class_list)):
    data_list=glob(f'../../data/stroke_mri/label/{class_list[i]}/**/')
    for j in tqdm(range(len(data_list))):
        label1=0
        label2=0
        file_list=glob(f'{data_list[j]}/*.png')
        file_list.sort()
        for k in range(len(file_list)):
            image=Image.open(file_list[k])
            image=np.array(image)
            if np.where(image==255)[0].shape[0]>0:
                if label1==0:
                    label1=k+1
                    label2=k+1
                else:
                    label2=k+1
        df.loc[count]=[class_list[i],os.path.basename(os.path.splitext(data_list[j])[0][:-1]),label1,label2]
        count+=1
df.to_csv('../../data/stroke_MRI_preprocessed/label.csv',index=False)

In [ ]:
df=pd.read_csv('../../data/stroke_MRI_preprocessed/label.csv')
flie_path='../../data/stroke_MRI_preprocessed/'
save_path='../../data/2D_MRI/'

class_list=['ADC','DWI']
for i in range(len(class_list)):
    data_list=glob(f'../../data/stroke_MRI_preprocessed/{class_list[i]}/Normal/**')
    for j in tqdm(range(len(data_list))):
        file_list=glob(f'{data_list[j]}/*.png')
        file_list.sort()
        k=random.randint(10,len(file_list)-10)
        image_path=file_list[k]
        create_dir(f'{save_path}{class_list[i]}/Normal/')
        shutil.copy(image_path,f'{save_path}{class_list[i]}/Normal/{os.path.basename(os.path.splitext(data_list[j])[0][:-1])}.png')
    
for i in tqdm(range(len(df))):
    try:
        file_info=df.loc[i]
        class_path=file_info['class']
        data_path=file_info['data']
        label1=file_info['label1']
        label2=file_info['label2']
        label=int((label1+label2)/2)

        create_dir(f'{save_path}{class_path}/Ischemic/')
        shutil.copy(f'{flie_path}{class_path}/Ischemic/{data_path}/{str(label).zfill(3)}.png',f'{save_path}{class_path}/Ischemic/{data_path}.png')
    except:
        print(f'Error in {i}')
    
    

In [ ]:
input_folder = "../../data/stroke_MRI_preprocessed/image/ADC/ANO1_0078/"
output_nifti = "../../data/temp/input_mri.nii.gz"
output_brain_mask = "../../data/temp/brain_mask.nii.gz"
image_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".png")])
image_list = []
for filename in image_files:
    img_path = os.path.join(input_folder, filename)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # 흑백 이미지로 로드
    image_list.append(img)
image_stack = np.stack(image_list, axis=0)
nifti_image = nib.Nifti1Image(image_stack, affine=np.eye(4))
nib.save(nifti_image, output_nifti)
run(["hd-bet", "-i", output_nifti, "-o", output_brain_mask, "-device", "cuda"])
# 5️⃣ 마스크 파일 로드 후 뇌가 있는 슬라이스 찾기
brain_mask_nii = nib.load(output_brain_mask)
brain_mask_data = brain_mask_nii.get_fdata()  # Shape: (Z, H, W) -> (슬라이스 개수, 높이, 너비)

# 6️⃣ 뇌가 포함된 슬라이스 찾기
brain_slices = np.where(np.any(brain_mask_data > 0, axis=(1, 2)))[0]

if len(brain_slices) > 0:
    first_slice = brain_slices[0]  # 뇌가 처음 등장하는 슬라이스 번호
    last_slice = brain_slices[-1]  # 뇌가 마지막 등장하는 슬라이스 번호
    print(f"✅ 뇌가 포함된 슬라이스: {first_slice}번 ~ {last_slice}번")
else:
    print("⚠️ 뇌가 포함된 슬라이스가 없습니다!")

In [ ]:
image_stack.shape